# Считывание файла (Не в pandas)

In [1]:
docs_titles = {}
def one_line(line):
    data = line.split('\t', 1)
    doc_id = int(data[0])
    if len(data[1]) == 1:
        title = ''
    else:
        title = data[1]
    docs_titles[doc_id] = title[:-1] # без \n

In [2]:
with open('data/docs_titles.tsv', 'r') as f:
    l = f.readline()
    for line in f:
        one_line(line)

In [3]:
len(docs_titles)

28026

в качестве предобработки избавимся от:

- верхнего регистра
- не русских и не английских букв и не цифр
- не будем рассматривать the a ... стоп слова
- фильтруем слова по-умному: stemmer (лемматизация хуже)

In [4]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import numpy as np
import re

/home/simon/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [9]:
# import pymorphy2
# morph = pymorphy2.MorphAnalyzer()

In [5]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/simon/snap/jupyter/6/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
stop_words = set(stopwords.words(["russian", "english"]))
stemmerR = SnowballStemmer("russian")
stemmerE = SnowballStemmer("english")

In [32]:
titles_data = {}
def title_process(title_id):
    title = docs_titles[title_id].lower()
    tmp = [
#         morph.parse(x.lower())[0].normal_form for x in re.sub('[^0-9a-zа-я]', ' ', title).split()
        stemmerR.stem(stemmerE.stem(x)) for x in re.sub('[^0-9a-zа-я]', ' ', title).split()
        if not x in stop_words
    ]
    titles_data[title_id] = ' '.join([x for x in tmp])

In [33]:
for t in docs_titles:
    title_process(t)

In [34]:
# titles_data
len(titles_data)

28026

# Анализ контента страниц 

это нужно сделать параллельно

нужно сохранить распарсенные данные для упрощения работы

... слишком много данных, ощуение что одних заголовков должно хватить

In [12]:
from multiprocessing.dummy import Pool as ThreadPool 
from multiprocessing.dummy import Lock as ThreadLock 
from multiprocessing.dummy import Value as ThreadValue
import functools
import codecs
import pymorphy2
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
# !python3 -m pip install pymorphy2

In [ ]:
def parallel(func):
    mutex = ThreadLock()
    n_thread = ThreadValue('i',0)
    @functools.wraps(func)
    def wrapper(*args, **argv):
        result = func(*args, **argv)
        with mutex:
            nonlocal n_thread
            n_thread.value +=1
            print(f"\r{n_thread.value} ready",end ='',flush = True)
        return result
    return wrapper

In [ ]:
documents = {}
morph = pymorphy2.MorphAnalyzer()

@parallel
def parser(n):
#     lit=set('.,?!абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ')
    file_path = 'data/content/{}.dat'.format(n)
    f = codecs.open(file_path, encoding='utf-8')
    soup = BeautifulSoup(f, 'lxml')
    content = soup.find_all('div') #  для группировки контента
    result = []
    for txt in content:
        cnt = 0
        for s in txt.stripped_strings:
            if cnt > 66:
                break
            for w in s.split():
                if len(result) > 200:
                    break
                if w.lower() not in stop_words:
                    result.append(morph.parse(w.lower())[0].normal_form)
        #                 result.append(stemmerR.stem(t.lower()))
    documents[n] = ' '.join(result)
    f.close()

In [ ]:
# with ThreadPool(10) as pool: 
#     pool.map(parser, list(range(1,28027)))

In [ ]:
# result

In [ ]:
# documents

In [ ]:
# file_path = 'data/content/{}.dat'.format(6)#6265
# f = codecs.open(file_path, encoding='utf-8')
# soup = BeautifulSoup(f, 'lxml')
# content = soup.find_all('div')
# # print(content[0].text.split())
# # print(soup)
# for x in content[3].strings:
#     if re.search(r'\s', x):
#         continue
#     print(x.strip())
# # print(content[3])
# f.close()

# # re.searc(r'\s', x)

In [ ]:
from tqdm import tqdm

def parser_stopwords(texts_dict):
    words_dict = {}
    for ID, text in tqdm(texts_dict.items()):
        words_dict[ID] = ' '.join([word for word in text.split() if not word in stop_words])
    return words_dict

In [ ]:
documents_nsw = parser_stopwords(documents)

In [ ]:
# txt_list = list(documents_nsw.values())
df = pd.DataFrame.from_dict(
    documents_nsw, orient='index', columns=['txt']
)
df['ind']=df.index
df = df.sort_values('ind')
df = df.drop(['ind'], axis=1)
df.shape

In [ ]:
# df.to_csv('text_lemm.csv', encoding = 'utf_8_sig')

In [ ]:
# обратно в список
# a = pd.read_csv('texts/text_result_plus_stemm.csv')
# doc_to_text={}
# for i in range(1, 28027):
#     doc_to_text[i]=a.txt[i-1]

# Ищу фичи

In [13]:
from sklearn.preprocessing import StandardScaler

Cтандартизацию стоит применять при использование алгоритмов, которые основываются на измерении расстояний, например, k ближайших соседей или метод опорных векторов. Значит применяем

In [14]:
train_data = pd.read_csv('data/train_groups.csv')
# пререгоняем в словарь
#  по группам
traingroups = {}
for i in range(len(train_data)):
    tmp = train_data.iloc[i]
    doc_group = tmp['group_id']
    doc_id = tmp['doc_id']
    doc_target = tmp['target']
    title = titles_data[doc_id]
    if doc_group not in traingroups:
        traingroups[doc_group] = []
    traingroups[doc_group].append((doc_id, title, doc_target))

X_train = []
y_train = []
groups_train = []

# общие слова
for grp in traingroups:
    docs = traingroups[grp] #  все тексты из группы
    for k, (doc_id, title, doc_target) in enumerate(docs):
        y_train.append(doc_target)
        groups_train.append(grp)
        words = set(title.strip().split())
        common = []
        for j in range(len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, doc_target_j = docs[j]
            words_j = set(title_j.strip().split())
            common.append(len(words.intersection(words_j)))
        X_train.append(sorted(common)[0:20])

In [15]:
X_train = np.array(X_train)
y_train = np.array(y_train)
traingroups = np.array(groups_train)
print (X_train.shape, y_train.shape, traingroups.shape)


(11690, 20) (11690,) (11690,)


In [16]:
test_data = pd.read_csv('data/test_groups.csv')
testgroups = {}
for i in range(len(test_data)):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = titles_data[doc_id]
    if doc_group not in testgroups:
        testgroups[doc_group] = []
    testgroups[doc_group].append((doc_id, title))
    
X_test = []
groups_test = []
for grp in testgroups:
    docs = testgroups[grp]
    for k, (doc_id, title) in enumerate(docs):
        groups_train.append(grp)
        words = set(title.strip().split())
        common = []
        for j in range(len(docs)):
            if k == j:
                continue
            doc_id_j, title_j = docs[j]
            words_j = set(title_j.strip().split())
            common.append(len(words.intersection(words_j)))
        X_test.append(sorted(common)[:20])

In [17]:
X_test = np.array(X_test)
testgroups = np.array(groups_test)
print (X_test.shape, testgroups.shape)

(16627, 20) (0,)


In [18]:
scale_features_std = StandardScaler()
X_train=scale_features_std.fit_transform(X_train)
X_test=scale_features_std.fit_transform(X_test)
print(X_train.shape, X_test.shape)

(11690, 20) (16627, 20)


### Попробуем что то еще

knn) 
- tfidf хорошо себя зарекомендовал в 1 дз
- так же как и cos метрика
- для метрики используем стандартизатор


tree) 
- для деревъев себя лучше проявляет нормализатор


svc)
- тут опять же используется стандартизатор

In [85]:
from sklearn.metrics import pairwise_distances
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import Normalizer

In [44]:
class container:
    def __init__(self, titles, features):
        vec = TfidfVectorizer()
        titles[0] = '' #  нулевого нет
        self.doc_tfidf = vec.fit_transform([titles[i] for i in range(len(titles))])
        tmp = {}
        for i in titles.keys():
            tmp[i] = titles[i].split(' ')
        self.doc = tmp
        self.n_features = features
        self.metric = self.cosine
            
    def cosine(self, group):
        n = self.n_features //2
        X = np.ones(shape=(group.size, self.n_features), dtype=np.float)
        for i, title in enumerate(pairwise_distances(self.doc_tfidf[group], metric='cosine')):
            X[i, :n] = sorted(title)[1:n+1]
        X[:, n:] = np.mean(X[:, :n], axis=0)
#         X[:, :n] /= X[:, n:]
        return X
    
    def read(self, file, dbsc_eps=0.5, dbsc_min_samples=5):
        data = pd.read_csv(file)
        groups = data.groupby('group_id')
        if 'target' in data.columns:
            X = np.zeros(shape=(data.shape[0], self.n_features+1), dtype=np.float)
            y = np.zeros(shape=(data.shape[0], ), dtype=bool)
            group_ids = np.zeros(shape=(data.shape[0], ), dtype=int)
            i = 0
            for group_id, group_index in groups.groups.items():
                j = i + group_index.size # выделяю группу
                group = data.iloc[group_index]
                group_ids[i:j] = group_id
                clf = DBSCAN(eps=dbsc_eps, metric="cosine", min_samples=dbsc_min_samples) # крут
                res = clf.fit_predict(self.doc_tfidf[group.doc_id])
                for t in range(len(res)):
                    if res[t] == -1:
                        res[t] = 0
                    else:
                        res[t] = 1
                y[i:j] = group.target
                X[i:j,:-1] = self.metric(group.doc_id)
                X[i:j,-1] = res
                i = j
            return X, y, group_ids
        else:
            X = np.zeros(shape=(data.shape[0], self.n_features+1), dtype=np.float)
            pair_ids = np.zeros(shape=(data.shape[0], ), dtype=int)
            i = 0
            for group_id, group_index in groups.groups.items():
                j = i + group_index.size
                group = data.iloc[group_index]
                pair_ids[i:j] = group.pair_id
                clf = DBSCAN(0.5, metric="cosine", min_samples=8)
                res = clf.fit_predict(self.doc_tfidf[group.doc_id])
                for t in range(len(res)):
                    if res[t] == -1:
                        res[t] = 0
                    else:
                        res[t] = 1
                X[i:j,:-1] = self.metric(group.doc_id)
                X[i:j,-1] = res
                i = j
            return X, pair_ids

In [21]:
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import cross_val_score

# import warnings
# warnings.filterwarnings("ignore")

In [46]:
def cross_val_knn():
    cos_f = range(4,16,2)
    dbscan_f = [0, 1]
    fech = [4, 10]
    nebrs = [25] #range(5, 30, 5)
    f1 = []
    for c_f in tqdm(cos_f):
        for db_f in dbscan_f:
            for f in fech:
                c = container(titles_data, c_f)
                X, y, groups_train = c.read('data/train_groups.csv')
                scaler = StandardScaler()
                scaler.fit(X[:, :c_f])
                if db_f==1:
                    lft = scaler.transform(X[:,:c_f])
                    X = np.hstack((lft, X[:,c_f][:,np.newaxis]))
                else:
                    X = scaler.transform(X[:,:c_f])
                X = np.hstack((X, X_train[:,:f]))
                for n in nebrs:
                    model = KNeighborsClassifier(n)
                    f1.append([cross_val_score(model, X, y, cv=10,scoring = 'f1', n_jobs=-1).mean(),c_f, db_f, f, X.shape, n])
    return f1
                    
f = cross_val_knn()
# c = container(titles_data, 8)
# c.read('data/train_groups.csv')

100%|██████████| 6/6 [01:21<00:00, 13.53s/it]


In [23]:
# score, c_f, db_f, f, X.shape, n
max(f,key=lambda x: x[0])

[0.7478353004768601, 10, 0, 10, (11690, 20), 25]

In [48]:
# score, c_f, db_f, f, X.shape, n
# max(f,key=lambda x: x[0])

In [50]:
# f

In [43]:
# [titles_data[i] for i in range(1, len(titles_data))]
# titles_data[1]

In [51]:
from sklearn.ensemble import RandomForestClassifier

In [86]:
model = 0
X = 0
y = 0
def cross_val_randomfor():
    c_f = 10
    min_samples_s = [ 4, 5]
    min_samples_l = [ 4, 5]
    n_esti = range(60, 80, 10)
    dbscan_f = [0, 1]
    db_f = 0
    f = 10
    fech = [4, 5]
    crit = ['gini'] #range(5, 30, 5)
    f1 = []
    for ss in tqdm(min_samples_s):
        for sl in min_samples_l:
            for cr in crit:
                c = container(titles_data, c_f)
                X, y, groups_train = c.read('data/train_groups.csv')
                scaler = Normalizer()
                scaler.fit(X[:, :c_f])
                if db_f==1:
                    lft = scaler.transform(X[:,:c_f])
                    X = np.hstack((lft, X[:,c_f][:,np.newaxis]))
                else:
                    X = scaler.transform(X[:,:c_f])
                X = np.hstack((X, X_train[:,:f]))
                for n in n_esti:
                    model = RandomForestClassifier(n_estimators=n, criterion=cr, min_samples_split=ss, min_samples_leaf=sl)
                    f1.append([cross_val_score(model, X, y,cv=10,scoring = 'f1').mean(),cr, ss, sl, X.shape, n])
    return f1
                    
f = cross_val_randomfor()

100%|██████████| 2/2 [01:23<00:00, 41.52s/it]


In [83]:
# c = container(titles_data, 10)
# X, y, groups_train = c.read('data/train_groups.csv')
# scaler = StandardScaler()
# scaler.fit(X[:, :10])
# model = RandomForestClassifier(n_estimators=80)
# X = np.hstack((X, X_train[:,:10]))
# cross_val_score(model, X, y, cv=10,scoring = 'f1', n_jobs=-1).mean()

In [87]:
max(f,key=lambda x: x[0])

[0.7150423033797212, 'gini', 5, 4, (11690, 20), 70]

In [88]:
from sklearn.svm import SVC

In [94]:
def cross_val_svm():
    c_f = 10
    С_reg = [0.5, 1.0, 2.0]
    kernel = ['linear', 'poly', 'rbf', 'sigmoid']
#     min_samples_l = [ 4, 5]
    n_esti = range(60, 80, 10)
    dbscan_f = [0, 1]
    db_f = 0
    f = 10
    fech = [4, 5]
    crit = ['gini'] #range(5, 30, 5)
    f1 = []
    for k in tqdm(kernel):
        for cr in С_reg:
            c = container(titles_data, c_f)
            X, y, groups_train = c.read('data/train_groups.csv')
            scaler = StandardScaler()
            scaler.fit(X[:, :c_f])
            if db_f==1:
                lft = scaler.transform(X[:,:c_f])
                X = np.hstack((lft, X[:,c_f][:,np.newaxis]))
            else:
                X = scaler.transform(X[:,:c_f])
            X = np.hstack((X, X_train[:,:f]))
            for n in n_esti:
                model = SVC(kernel=k, C=cr)
                f1.append([cross_val_score(model, X, y,cv=10,scoring = 'f1').mean(),k, X.shape, cr])
    return f1
                    
f = cross_val_svm()

100%|██████████| 4/4 [09:12<00:00, 138.21s/it]


In [96]:
max(f,key=lambda x: x[0])

[0.7311443558677078, 'rbf', (11690, 20), 1.0]

# Учимся

...

In [35]:
c_f = 10
n = 25
f = 10 #fech а надо ли?
c = container(titles_data, c_f)
X_tr, y_tr, groups_tr = c.read('data/train_groups.csv')
scaler = StandardScaler()
scaler.fit(X_tr[:, :c_f])
X_tr = scaler.transform(X_tr[:,:c_f])
# X_tr = np.hstack((X_tr, X_train[:,:f]))
model = KNeighborsClassifier(n)
model.fit(X_tr, y_tr)

KNeighborsClassifier(n_neighbors=25)

In [36]:
X_tr.shape

(11690, 20)

In [40]:
c = container(titles_data, c_f)
X_te, pair_ids = c.read('data/test_groups.csv')
scaler = StandardScaler()
scaler.fit(X_te[:, :c_f])
X_te = scaler.transform(X_te[:, :c_f])
X_te = np.hstack((X_te, X_test[:,:f]))
# X_te = np.hstack((X_te, X_test))

In [41]:
X_te.shape

(16627, 20)

In [42]:
predict = model.predict(X_te)

In [43]:
with open('predict.csv', 'w') as f:
    print('pair_id', 'target', file=f, sep=',')
    for pair_id, target in zip(pair_ids, predict):
        print(pair_id, int(target), sep=',', file=f)

## Еще хотелось бы попробовать бустинг и голосование алгоритмов